Trying to make a FCN to predict dihedral angles. The data is from https://github.com/aqlaboratory/proteinnet casp7, text based.

In [1]:
import numpy as np
import torch
import imp
import re
import matplotlib.pyplot as plt

In [2]:
aa_id_dict = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7,
              'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14, 
              'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}


First define some helper functions.

In [3]:
def aa_to_onehot(aa_str, aa_to_nr, mask=None):
    """
    Onehot encode an amino acid string using a letter to number dictionary.
    The mask (from proteinnet files) is used to remove residues missing atoms from the primary sequence.
    """
    if mask!=None:
        mask_ind = np.asarray([x=='+' for x in mask])*1
        mask_ind = np.nonzero(mask_ind)
        aa_str = "".join([aa_str[x] for x in mask_ind[0]]) # the mask indices are a list in a list
    init_array = np.zeros( (len(aa_to_nr.keys()), len(aa_str)) )
    for i,j in enumerate(aa_str):
        init_array[aa_to_nr[j], i] = 1
    return(init_array)

def filter_primary(aa_str, mask):
    '''
    Need this later on for writing pdb files, but should probably just use it at the start...
    '''
    mask_ind = np.asarray([x=='+' for x in mask])*1
    mask_ind = np.nonzero(mask_ind)
    aa_str = "".join([aa_str[x] for x in mask_ind[0]]) # the mask indices are a list in a list
    return(aa_str)

In [4]:
def read_proteinnet_file(file, stop_at=1000, verbose=True):
    """
    Read a proteinnet file. Will also filter the 3D coordinates using the mask
    and remove proteins with chainbreaks and missing structures. 
    A certain number of proteins are loaded.
    """
    protein_dict = {}
    with open(file) as input:
        lines = input.readlines()
        curr_id = None
        for i, line in enumerate(lines):
            line = line.strip()
            if line == '[ID]':
                curr_id = lines[i+1].strip()
                protein_dict[curr_id] = {}
                protein_dict[curr_id]['primary'] = lines[i+3].strip()
            if line == '[TERTIARY]':
                coords = []
                for j in range(3):
                    coords.append(np.fromstring(lines[i+j+1], sep='\t'))
                protein_dict[curr_id]['tertiary'] = np.array(coords)
            if line == '[MASK]':
                protein_dict[curr_id]['mask'] = lines[i+1].strip()
            if len(protein_dict.keys()) >= stop_at:
                break
    filter_seqs(protein_dict, verbose)
    return(protein_dict)

According to the proteinnet documentation the protein coordinates should be stored as a sequence of 3x3 arrays corresponding to the coordinates of N, CA and C for each residue.

In [5]:
def filter_coords(coords, mask):
    """
    Filter atomic coordinates (for residues missing some) using the mask.
    coords = 3*(N*3) array, since each residue has N, CA and C
    """
    mask = np.array([x=='+' for x in mask])
    mask_stretched = np.repeat(mask, 3)
    coords_filt = coords[:, mask_stretched]
    return(coords_filt)

def filter_seqs(protein_dict, verbose):
    re_chainbreak = re.compile("\-*\+*\+\-+\+\+*\-*") # match for a mask with internal chainbreaks
    keys_to_remove = {}
    for key in protein_dict.keys():
        if len(protein_dict[key].keys()) < 3:
            keys_to_remove[key] = ' missing structure, removing...'
            continue
        mask = protein_dict[key]['mask']
        if re_chainbreak.search(mask):
            keys_to_remove[key] = " has a chainbreak, removing..."
        else:
            coords = protein_dict[key]['tertiary']
            coords_filt = filter_coords(coords, mask)
            protein_dict[key]['tertiary'] = coords_filt
    for key in keys_to_remove.keys():
        if verbose:
            print(key, keys_to_remove[key])
        del protein_dict[key]

def new_dihedral(p0, p1, p2, p3):
    """Praxeolitic formula
    1 sqrt, 1 cross product
    
    copied from 
    https://stackoverflow.com/questions/20305272/
    dihedral-torsion-angle-from-four-points-in-cartesian-coordinates-in-python"""

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    #return np.degrees(np.arctan2(y, x))
    return np.arctan2(y, x)

def calc_angles(coords):
    '''
    Basically loops over the coordinates and calculates the dihedral angles.
    There should be ways to do this with vectorized computations but I don't know how.
    '''
    N = coords.shape[1]
    angles_all = []
    for i in range(0, N-3, 3):
        psi = new_dihedral(*[coords[:, x] for x in range(i, i+4)])
        omega = new_dihedral(*[coords[:, x] for x in range(i+1, i+5)])
        phi = new_dihedral(*[coords[:, x] for x in range(i+2, i+6)])
        angles = [psi, omega, phi]
        angles_all.append(angles)
    return(np.array(angles_all))

Load some proteins.

In [135]:
train_proteins = read_proteinnet_file('/Users/Deathvoodoo/big_folders_docs/data/casp7/training_70', 10000, verbose=False)
val_proteins = read_proteinnet_file('/Users/Deathvoodoo/big_folders_docs/data/casp7/validation', 1000, verbose=False)

Use the function below to transform the sequences and coordinates into one-hot encoding and dihedral angles. The easiest thing to do would be to pad everything to the length of the longest protein and put it in one tensor, but this has consequences for the output of the network later (nonzero outputs at places that should be zero).
<br>
<br>
Note that since I don't have access to powerful compute resources I'm only running with a small number of proteins for training and validation.

In [136]:
def get_onehot_angles(protein_dict, pad, max_length=1500):
    """
    Get onehot sequence and calculate angles from 3d coords for the protein dictionary.
    Sometimes the angle calculation will produce nans, we remove those sequences.
    If pad==True we will numbers as tensors, otherwise lists, along with remaining ids.
    """
    remaining_ids = list(protein_dict.keys())
    sequence_list = []
    angle_list = []
    for key in protein_dict.keys():
        sequence = aa_to_onehot(protein_dict[key]['primary'], aa_id_dict, protein_dict[key]['mask'])
        coords = protein_dict[key]['tertiary']
        angles = calc_angles(coords).T
        if pad:
            if max_length-sequence.shape[0] < 0:
                print(key, ' exceeds max length: ', max_length, ' , skipping...')
                continue
            else:
                # pad the width, current shape is C*W
                sequence_padded = np.pad(
                    sequence, pad_width = ((0,0), (0, max_length-sequence.shape[1])), 
                    constant_values = 0)
                sequence_list.append(sequence_padded)
                angles_padded = np.pad(
                    angles, pad_width = ((0,0), (0, max_length-sequence.shape[1])),
                    constant_values = 0)
                angle_list.append(angles_padded)
        else: 
            angle_list.append(angles)
            sequence_list.append(sequence)
    N = len(sequence_list)
    have_nans = []
    for i,j in enumerate(angle_list):
        if np.isnan(j).any():
            have_nans.append(i)
    inds_to_keep = np.arange(N)[np.logical_not(np.isin(np.arange(N), have_nans))].tolist()
    
    sequence_list = [sequence_list[i] for i in inds_to_keep]
    angle_list = [angle_list[i] for i in inds_to_keep]
    remaining_ids = [remaining_ids[i] for i in inds_to_keep]
    coord_list = [protein_dict[rem_id]['tertiary'] for rem_id in remaining_ids]
    if pad:
        sequence_tensor = torch.tensor(sequence_list).float()
        sequence_tensor = sequence_tensor #.unsqueeze(1)
        angle_tensor = torch.tensor(angle_list).float()
        angle_tensor = angle_tensor #.unsqueeze(1)
        return([sequence_tensor, angle_tensor, remaining_ids])
    else:
        for i in range(len(sequence_list)):
            ith_seq = (torch.tensor(sequence_list[i]).float()).unsqueeze(0)
            sequence_list[i] = ith_seq #.unsqueeze(0)
            ith_angles = (torch.tensor(angle_list[i]).float()).unsqueeze(0)
            angle_list[i] = ith_angles #.unsqueeze(0)
        return([sequence_list, coord_list, angle_list, remaining_ids])

In [137]:
train_seqs, train_coords, train_angles, train_ids_remain = get_onehot_angles(train_proteins, pad=False)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


In [138]:
val_seqs, val_coords, val_angles, val_ids_remain = get_onehot_angles(val_proteins, pad=False)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


We will also calculate sin and cos to transform the angles into better values for the squared loss error function.

In [140]:
def get_sin_cos(array, as_torch_tensor=True):
    sin = np.sin(array)
    cos = np.cos(array)
    sin_cos = np.concatenate([sin, cos], axis=1)
    if as_torch_tensor:
        sin_cos = torch.tensor(sin_cos)
    return(sin_cos)

def back_to_angle(sin_cos_array, n_angles=3):
    angles = []
    for i in range(n_angles):
        angles_i = np.arctan2(sin_cos_array[:, i, :], sin_cos_array[:, n_angles+i, :])
        # remember that we input sin first due to atan2 definition
        angles_i = angles_i[:, np.newaxis, :]
        angles.append(angles_i)
    angles_arr = np.concatenate(angles, 1)
    return(angles_arr)

In [141]:
train_angles_sin_cos = [get_sin_cos(i) for i in train_angles]
val_angles_sin_cos = [get_sin_cos(i) for i in val_angles]

Reading the proteins and transforming the coords to angles etc. takes forever, so we just load some of them and then pickle them for later.

In [154]:
import pickle
train_data_proc = {'train_seqs':train_seqs, 'train_coords':train_coords,
                   'train_angles_sin_cos': train_angles_sin_cos, 'train_ids_after_filt':train_ids_remain}

val_data_proc = {'val_seqs':val_seqs, 'val_coords':val_coords,
                   'val_angles_sin_cos': val_angles_sin_cos, 'val_ids_after_filt':val_ids_remain}

In [157]:
train_file = open('protein_fcn_data_test/train_data_proc', 'wb')
pickle.dump(train_data_proc, train_file)
train_file.close()

In [158]:
val_file = open('protein_fcn_data_test/val_data_proc', 'wb')
pickle.dump(val_data_proc, val_file)
val_file.close()

In [161]:
asdf = open('protein_fcn_data_test/val_data_proc', 'rb')
test = pickle.load(asdf)
asdf.close()
